# Experiment plotting

This notebook contains the code for plotting results for the different experiments.
When run for the first time for a town, condensed summary files are being created which strongly speed up subsequent generations of plots from the same summaries. It is possible to create the plots only from the condensed summaries located in 'summaries/condensed_summaries'.
Note that this works for all experiments but the Rt plots, which still require the full summary file.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
import pickle
from lib.measures import *
from lib.experiment import Experiment, Plot, Result, get_properties, load_summary_list, load_summary
from lib.data import collect_data_from_df
from lib.calibrationSettings import calibration_lockdown_dates, calibration_start_dates, calibration_mob_paths
from lib.calibrationFunctions import get_calibrated_params, downsample_cases
import lib.plot as lib_plot
from lib.plot import Plotter
from lib.summary import load_condensed_summary

In [ ]:
places = [('CH', 'TI')]
ymax = {'CH' : {'TI': 100, 'BE': 40000, 'JU': 25000},
        'GER': {'TU': 300, 'KL': 2000, 'RH': 3500}}

## ROC

In [ ]:
def plot_roc(country, area, p_adoption):
    # Format: {`filename for plot` : `list of `Plot` objects`}
    plots = {
         f'beacon-environment-{country}-{area}' : [
            Plot(label='tracing-stats', 
                path=(f'beacon-environment-{country}-{area}/'
                      f'beacon-environment-{country}-{area}-p_adoption={p_adoption}-x=8.0.pk')),
        ],
    }
    
    for plot_filename, plots in plots.items():
        # extract plot info
        labels = get_properties(plots, 'label')
        paths = get_properties(plots, 'path')
        
        #results = load_summary_list(paths)
        #summaries = get_properties(results, 'summary')
    
        # plots
        plotter = Plotter()

        plotter.plot_roc_curve(
            titles=labels,
            summaries=None,
            paths=paths,
            filename=plot_filename, 
            action='isolate',
#             figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
            figsize=(4, 2), 
            figformat='neurips-double')
        
p_adoption = 0.5
        
for country, area in places:
    plot_roc(country, area, p_adoption)

In [ ]:
# Crop pdf margins
# !bash crop_pdfs.sh plots/beacon-environment-*.pdf


# Beacons


In [ ]:
def plot_beacon_proportion(country, area, ymax):
    # Format: {`filename for plot` : `list of `Plot` objects`}
    plots = {
        f'beacons-proportion-{country}-{area}' : [
            Plot(label='P2P',
                path=(f'beacon-baseline-{country}-{area}/'
                      f'beacon-baseline-{country}-{area}-p_adoption=0.25.pk')),
            
#             Plot(label='Beacon at top 5\% sites',
#                 path=(f'beacon-sparse-locations-{country}-{area}/'
#                       f'beacon-sparse-locations-{country}-{area}-p_adoption=0.25-p_recall=1.0-beacon_mode=visit_freq-beacon_proportion=0.05.pk')),
#             Plot(label='Beacon at top 10\% sites',
#                 path=(f'beacon-sparse-locations-{country}-{area}/'
#                       f'beacon-sparse-locations-{country}-{area}-p_adoption=0.25-p_recall=1.0-beacon_mode=visit_freq-beacon_proportion=0.1.pk')),
            Plot(label='Beacon at top 25\% sites',
                path=(f'beacon-sparse-locations-{country}-{area}/'
                      f'beacon-sparse-locations-{country}-{area}-p_adoption=0.25-p_recall=1.0-beacon_mode=visit_freq-beacon_proportion=0.25.pk')),
            Plot(label='Beacon at top 50\% sites',
                path=(f'beacon-sparse-locations-{country}-{area}/'
                      f'beacon-sparse-locations-{country}-{area}-p_adoption=0.25-p_recall=1.0-beacon_mode=visit_freq-beacon_proportion=0.5.pk')),
            Plot(label='Beacon at all sites',
                path=(f'beacon-sparse-locations-{country}-{area}/'
                      f'beacon-sparse-locations-{country}-{area}-p_adoption=0.25-p_recall=1.0-beacon_mode=visit_freq-beacon_proportion=1.0.pk')),
            
        
        ]
    }

    for plot_filename, plots in plots.items():

        # extract plot info
        labels = get_properties(plots, 'label')
        paths = get_properties(plots, 'path')

        # plots
        plotter = Plotter()

        plotter.compare_total_infections(
            paths, 
            titles=labels, 
            filename=plot_filename, 
            show_single_runs=False,
#             figsize=lib_plot.FIG_SIZE_FULL_PAGE_TRIPLE_ARXIV, 
            figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
            figformat='neurips-double',
            legend_is_left=True,
            lockdown_label_y=1000,
            ymax=ymax[country][area],
            x_axis_dates=False,
            xtick_interval=3,
            errorevery=14)

In [ ]:
plot_beacon_proportion(country, area, ymax)

In [ ]:
def plot_beacon_manual_tracing(country, area, ymax):
    # Format: {`filename for plot` : `list of `Plot` objects`}
    plots = {
        f'beacons-manual-tracing-{country}-{area}' : [
            Plot(label='P2P, 10\% compliance',
                path=(f'beacon-baseline-{country}-{area}/'
                      f'beacon-baseline-{country}-{area}-p_adoption=0.1.pk')),
            Plot(label='P2P, 25\% compliance',
                path=(f'beacon-baseline-{country}-{area}/'
                      f'beacon-baseline-{country}-{area}-p_adoption=0.25.pk')),
            Plot(label='P2P, 50\% compliance',
                path=(f'beacon-baseline-{country}-{area}/'
                      f'beacon-baseline-{country}-{area}-p_adoption=0.5.pk')),
            Plot(label='P2P, 100\% compliance',
                path=(f'beacon-baseline-{country}-{area}/'
                      f'beacon-baseline-{country}-{area}-p_adoption=1.0.pk')),
                
            
            Plot(label='Beacon, 10\% compliance',
                path=(f'beacon-manual-tracing-{country}-{area}/'
                      f'beacon-manual-tracing-{country}-{area}-p_adoption=0.1-p_recall=1.0-beacon_proportion=1.0-p_willing_to_share=1.0.pk')),
            Plot(label='Beacon, 25\% compliance',
                path=(f'beacon-manual-tracing-{country}-{area}/'
                      f'beacon-manual-tracing-{country}-{area}-p_adoption=0.25-p_recall=1.0-beacon_proportion=1.0-p_willing_to_share=1.0.pk')),
            Plot(label='Beacon, 50\% compliance',
                path=(f'beacon-manual-tracing-{country}-{area}/'
                      f'beacon-manual-tracing-{country}-{area}-p_adoption=0.5-p_recall=1.0-beacon_proportion=1.0-p_willing_to_share=1.0.pk')),
            Plot(label='Beacon, 100\% compliance',
                path=(f'beacon-manual-tracing-{country}-{area}/'
                      f'beacon-manual-tracing-{country}-{area}-p_adoption=1.0-p_recall=1.0-beacon_proportion=1.0-p_willing_to_share=1.0.pk')),    
        
        ]
    }

    for plot_filename, plots in plots.items():

        # extract plot info
        labels = get_properties(plots, 'label')
        paths = get_properties(plots, 'path')

        # plots
        plotter = Plotter()

        plotter.compare_total_infections(
            paths, 
            titles=labels, 
            filename=plot_filename, 
            show_single_runs=False,
#             figsize=lib_plot.FIG_SIZE_FULL_PAGE_TRIPLE_ARXIV, 
            figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
            figformat='neurips-double',
            legend_is_left=True,
            lockdown_label_y=1000,
            ymax=ymax[country][area],
            x_axis_dates=False,
            xtick_interval=3,
            errorevery=14)

In [ ]:
plot_beacon_manual_tracing(country, area, ymax)